In [3]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import lightgbm as lgb
from sklearn.metrics import f1_score
# import time
# print('sleeping')
# time.sleep(7200)
# print('sleep done =======================')
# load feats
train_x,test_x = [],[]
for feat in sorted(glob.glob('../feature/stacking_pkl_file/*')):
    if 'fm2.pkl' in feat or 'ligbm' in  feat :
        continue
    print('file path',feat)
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)


file path ../feature/stacking_pkl_file/2leve_lgb_stacking3.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/Capsule.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/capsule_all_shuffle.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/capsule_shuffle.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/cnn.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/mlp.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/mlp_shuffle.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/pseudo_rnn2.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/rnn7879.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/rnn_all_shuffle_all.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/rnn_cnn.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/rnn_pseudo1.pkl
(102277, 19) (102277, 19)
file path ../feature/stackin

In [4]:
for feat in sorted(glob.glob('../feature/other_features/features-vinson/*')):
    if 'fm2.pkl' in feat or 'w2v' in feat or 'lda' in feat or 'lsi' in feat:
        continue
    print('file path',feat)
    file=pd.read_csv(feat)
    a = file.iloc[:102277,:]
    b = file.iloc[102277:,:]
    a = a.values
    b = b.values
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)
from catboost import CatBoostRegressor

file path ../feature/other_features/features-vinson/bnb_prob_10w.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/lr_prob.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/lr_prob_0.778620.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/mnb_prob_10w.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/nn_ensemble_0.775597.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/svc_prob.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/svc_prob_0.778881.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/word_fasttext_0.760225.csv
(102277, 19) (102277, 19)
(102277, 418)


In [5]:
import pandas as pd
import numpy as np
train = pd.read_csv("../input/new_data/train_set.csv")
y=(train["class"]-1).astype(int)
del train
import gc
gc.collect()

/usr/local/lib/python3.5/dist-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


472

In [6]:
from keras.utils import np_utils
train_y=np_utils.to_categorical(y,num_classes=19)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
def self_f1_score(data, y_hat):
    y_true = data
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return f1_score(y_true, y_hat, average='macro')

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
import xgboost as xgb
def simple_ens(model_name,k=3,rnd=233,lr=0.05,c_bytree=0.9,s_sample=0.9):
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=rnd)
    test_pred, val_pred = np.zeros((102277,20)), np.zeros((102277,20))
    all_train_loss_l,all_val_loss_l = 0,0
    all_train_f1_l,all_val_f1_l = 0,0
    
    for i in range(19):
        val_loss_l,train_loss_l = 0,0
        val_f1_l,train_f1_l = 0,0
        fold_cnt = 0
        for train_index, test_index in kf.split(train_x,train_y[:,i]):
            # x,y
            curr_x,curr_y = train_x[train_index],train_y[train_index]
            hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
            d_test =xgb.DMatrix(test_x)
            
            d_train = xgb.DMatrix(curr_x, curr_y[:,i])
            d_valid = xgb.DMatrix(hold_out_x, hold_out_y[:,i])
            watchlist = [d_train, d_valid]
            watchlist = [(d_train, 'train'), (d_valid, 'valid')]
            model = lgb.train(params,
                      train_set=d_train,
                      num_boost_round=2000,
                      valid_sets=watchlist,
                      early_stopping_rounds=s_round,
                      verbose_eval=None)
        
            model.fit(curr_x, curr_y[:,i], eval_set=(hold_out_x, hold_out_y[:,i]), use_best_model=True, verbose=False)
            print(fold_cnt,'fold: ',end='')
            fold_cnt += 1
            train_pred = model.predict(curr_x)
            val_pred[test_index, i] = model.predict(hold_out_x)
            curr_train_loss = log_loss(curr_y[:,i],train_pred)
            curr_val_loss = log_loss(hold_out_y[:,i],val_pred[test_index, i])

            curr_train_f1 = self_f1_score(curr_y[:,i],train_pred)
            curr_val_f1 = self_f1_score(hold_out_y[:,i],val_pred[test_index, i])

            print('ls',curr_train_loss,curr_val_loss,'f1',curr_train_f1,curr_val_f1)
            val_loss_l += curr_val_loss
            train_loss_l += curr_train_loss
            val_f1_l += curr_val_f1
            train_f1_l += curr_train_f1
            curr_test_pred = model.predict(d_test)
            test_pred[:,i] += curr_test_pred
            
        # avg k fold
        train_loss_l = train_loss_l/k
        val_loss_l = val_loss_l/k
        train_f1_l = train_f1_l/k
        val_f1_l = val_f1_l/k
        print('this class avg train',train_loss_l,'avg val',val_loss_l)
        print('this class f1 train',train_f1_l,'f1 val',val_f1_l)
        
        
        # avg 6 class
        all_train_loss_l += train_loss_l/19
        all_val_loss_l += val_loss_l/19
        all_train_f1_l += train_f1_l/19
        all_val_f1_l += val_f1_l/19
        print('========================')
    test_pred = test_pred/k
    print('all loss avg',all_train_loss_l,all_val_loss_l)
    print('all f1 avg',all_train_f1_l,all_val_f1_l)
    print('=======================================================')
    return val_pred, test_pred

print('done')